In [1]:
# Imports
import os
import getpass

import numpy

import nipype
import nibabel
from nilearn.plotting import plot_roi
from nilearn import image
from nipype.interfaces.io import BIDSDataGrabber
from bids.layout import BIDSLayout
from nipype.pipeline import engine as pe

from matplotlib import pyplot as plt
from os.path import join
from bids.tests import get_test_data_path

210219-16:59:30,40 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
user = getpass.getuser()

if user == 'ari':
    bids_ds_path = '/Volumes/SSD-Bercows/Joan/research/0_data/Sample_Multi-Modal_BIDS_dataset/ds-sample/'

if user == 'andres':
    bids_ds_path = '/home/andres/Desktop/BIDS/Datos/ds-sample'
    #deriv_dir = 

deriv_cmp_dir = join(bids_ds_path, "derivatives", "cmp")
deriv_nipype_dir = join(bids_ds_path, "nipype", "sub-01", "ses-01")

In [3]:
if user == 'ari':
    !tree -L 4 /Volumes/SSD-Bercows/Joan/research/0_data/Sample_Multi-Modal_BIDS_dataset/ds-sample/

if user == 'andres':
    !tree -L 4 /home/andres/Desktop/BIDS/Datos/ds-sample

/home/andres/Desktop/BIDS/Datos/ds-sample
├── CHANGES
├── code
│   ├── license.txt
│   ├── ref_anatomical_config.ini
│   ├── ref_anatomical_config.json
│   ├── ref_diffusion_config.ini
│   ├── ref_diffusion_config.json
│   ├── ref_fMRI_config.ini
│   └── ref_fMRI_config.json
├── dataset_description.json
├── derivatives
│   ├── cmp
│   │   ├── dataset_description.json
│   │   └── sub-01
│   │       └── ses-01
│   ├── freesurfer
│   │   ├── dataset_description.json
│   │   ├── fsaverage
│   │   │   ├── label
│   │   │   ├── mri
│   │   │   ├── mri.2mm
│   │   │   ├── scripts
│   │   │   ├── surf
│   │   │   └── xhemi
│   │   └── sub-01_ses-01
│   │       ├── label
│   │       ├── mri
│   │       ├── scripts
│   │       ├── stats
│   │       ├── surf
│   │       ├── tmp
│   │       ├── touch
│   │       └── trash
│   └── nipype
│       ├── dataset_description.json
│       └── sub-01
│           └── ses-01
├── participants.tsv
├── README
└── sub-01
    └── ses-01
        ├── anat
        │

In [4]:
layout = BIDSLayout(bids_ds_path)
layout.add_derivatives(deriv_cmp_dir)
layout.add_derivatives(deriv_nipype_dir)

/home/andres/miniconda3/envs/py37cmp-gui/lib/python3.7/site-packages/bids/layout/layout.py:774: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations (['/home/andres/Desktop/BIDS/Datos/ds-sample/nipype/sub-01/ses-01']). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a dataset_description.json file in derivatives datasets).
  "datasets).".format(paths))


In [5]:
# Import Node and Function module
from nipype import Node, Function, Workflow
import nipype.interfaces.io as nio
from nipype.interfaces.utility import IdentityInterface

# Create a small example function
def overlay_t1w_parcellation(subject, parcellation, layout):
    t1w_path = layout.get(datatype="anat", suffix="T1w", subject=subject, return_type="file", extension=["nii", "nii.gz"])
    roi_path = layout.get(datatype="anat", scope="derivatives", suffix="atlas", subject=subject, return_type="file", extension="nii.gz")
    
    t1w_image = nibabel.load(t1w_path[0])
    
    roi_path_selected = [elem for elem in roi_path if parcellation in elem]
    print(roi_path_selected)
    roi_image = nibabel.load(roi_path_selected[0])
    
    fig = plot_roi(roi_image, t1w_image)
    return fig

overlay = Node(Function(input_names=["subject", "parcellation", "layout"],
                       output_names=["fig_output"],
                       function=overlay_t1w_parcellation),
                       name='overlay_fig')

ii = IdentityInterface(fields=['subject', 'parcellation', 'layout'], mandatory_inputs=True)
inputNode = Node(ii, name='input_node')

In [6]:
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory = deriv_nipype_dir
workflow = Workflow("overlay_workflow")
#workflow.connect(overlay, 'subject_id', datasink, 'container')

In [7]:
workflow.connect([(inputNode, overlay,
                   [('subject', 'subject'),
                    ('parcellation','parcellation'),
                    ('layout', 'layout')]
                  )])
workflow.connect(overlay, 'fig_output', datasink, 'fig_input')

In [8]:
inputNode.inputs.subject = "01"
inputNode.inputs.parcellation = "scale3"
inputNode.inputs.layout = layout

In [9]:
workflow.run()

TypeError: can't pickle _thread._local objects

In [ ]:
datasink